In [0]:
from google.colab import drive
import numpy as np
import math
import pandas as pd
import os
import pickle
import sklearn
from sklearn.externals import joblib
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error 

In [0]:
drive.mount('/content/drive')
path = os.getcwd()
print(path)
df = pd.read_csv(path + '/drive/My Drive/ML Project/Test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


In [0]:
df

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,Origin,Dest,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,CRSDepTimeRound,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,airport
0,2016-01-01,1,2016,1,1,741.0,0.0,745,0.0,SEA,JFK,1610.0,1602,0.0,8.0,800,7,108,113,0.0,10,1030,0,23,8,34,29,65,2016-01-01,800,SEA
1,2016-01-02,1,2016,1,2,737.0,0.0,745,0.0,SEA,JFK,1613.0,1602,0.0,11.0,800,7,118,113,0.0,10,1024,0,25,8,37,33,63,2016-01-02,800,SEA
2,2016-01-03,1,2016,1,3,743.0,0.0,745,0.0,SEA,JFK,1547.0,1602,0.0,0.0,800,6,83,113,0.0,10,1018,0,20,7,32,28,64,2016-01-03,800,SEA
3,2016-01-04,1,2016,1,4,737.0,0.0,745,0.0,SEA,JFK,1551.0,1602,0.0,0.0,800,5,9,266,0.2,4,1010,93,33,8,35,31,94,2016-01-04,800,SEA
4,2016-01-05,1,2016,1,5,708.0,0.0,710,0.0,SEA,JFK,1524.0,1527,0.0,0.0,700,5,62,266,0.1,6,1001,100,35,10,38,35,91,2016-01-05,700,SEA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851431,2017-12-17,4,2017,12,17,1024.0,0.0,1030,0.0,ORD,IAH,1314.0,1326,0.0,0.0,1100,14,140,260,0.0,9,1020,100,31,16,32,27,97,2017-12-17,1100,ORD
1851432,2017-12-17,4,2017,12,17,1148.0,0.0,1156,0.0,LAX,EWR,1939.0,2017,0.0,0.0,1200,17,31,113,0.0,10,1014,0,25,19,58,56,27,2017-12-17,1200,LAX
1851433,2017-12-17,4,2017,12,17,739.0,0.0,737,2.0,IAH,DEN,904.0,909,0.0,0.0,800,7,24,122,0.0,10,1020,100,47,9,49,48,91,2017-12-17,800,IAH
1851434,2017-12-17,4,2017,12,17,1632.0,0.0,1638,0.0,DFW,DEN,1727.0,1747,0.0,0.0,1700,8,171,122,0.0,10,1019,95,44,13,50,48,78,2017-12-17,1700,DFW


In [0]:
train = pd.read_csv(path + "/drive/My Drive/ML Project/Train.csv")
test = pd.read_csv(path + "/drive/My Drive/ML Project/Test.csv")
print(train.shape, test.shape)

(1388577, 30) (462859, 30)


In [0]:
train = train[train['ArrDel15'] != 0]
test = test[test['ArrDel15'] != 0]
train_labels = train['ArrDelayMinutes']
test_labels = test['ArrDelayMinutes']
print(train.shape, test.shape)

(290876, 30) (97182, 30)


In [0]:
train

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,Origin,Dest,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,CRSDepTimeRound,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time
0,2017-04-04,2,2017,4,4,2143.0,0.0,2155,0.0,LAS,JFK,542.0,555,0.0,0.0,2200,11,65,113,0.0,10,1018,100,18,12,63,64,17,2017-04-04,2200
1,2017-07-09,3,2017,7,9,1605.0,0.0,1608,0.0,PHX,ATL,2243.0,2255,0.0,0.0,1600,17,268,113,0.0,10,1010,0,55,19,106,106,18,2017-07-09,1600
2,2017-02-04,1,2017,2,4,1151.0,1.0,1110,41.0,ORD,MCO,1518.0,1449,1.0,29.0,1100,19,196,122,0.0,10,1027,93,13,29,24,12,67,2017-02-04,1100
3,2016-10-29,4,2016,10,29,1733.0,0.0,1735,0.0,EWR,DEN,1940.0,2000,0.0,0.0,1800,19,221,119,0.0,10,1013,66,48,28,61,59,66,2016-10-29,1800
4,2016-07-01,3,2016,7,1,1803.0,1.0,1510,173.0,EWR,MCO,2053.0,1801,1.0,172.0,1500,23,162,353,1.1,9,1012,69,68,34,82,82,59,2016-07-01,1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1388572,2017-09-25,3,2017,9,25,2103.0,0.0,2100,3.0,ORD,IAH,2333.0,2340,0.0,0.0,2100,10,151,113,0.0,10,1013,5,63,12,85,85,48,2017-09-25,2100
1388573,2016-06-13,2,2016,6,13,1155.0,0.0,1200,0.0,LAX,SFO,1418.0,1325,1.0,53.0,1200,15,235,119,0.1,8,1019,79,58,18,65,63,84,2016-06-13,1200
1388574,2016-08-12,3,2016,8,12,1845.0,0.0,1850,0.0,PHX,LAS,1944.0,1959,0.0,0.0,1900,15,282,113,0.0,10,1007,0,64,17,92,92,32,2016-08-12,1900
1388575,2017-11-28,4,2017,11,28,809.0,0.0,810,0.0,PHX,LAX,824.0,840,0.0,0.0,800,5,149,113,0.0,10,1015,0,39,7,56,56,52,2017-11-28,800


In [0]:
regressor = LinearRegression()  
regressor.fit(train.drop(['Origin','Dest','FlightDate','date', 'ArrDel15','ArrDelayMinutes', 'ArrTime'], axis = 1), train_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
y_pred = regressor.predict(test.drop(['Origin','Dest','FlightDate','date', 'ArrDel15','ArrDelayMinutes', 'ArrTime'], axis = 1))
print(mean_absolute_error(test_labels, y_pred))
print(math.sqrt(mean_squared_error(test_labels, y_pred)))
df_lr = pd.DataFrame({'Actual': test_labels, 'Predicted': y_pred})
print(df_lr)
filename = path + '/drive/My Drive/ML Project/regressor_model_lr.sav'
pickle.dump(regressor, open(filename, 'wb'))



12.327299463100042
17.68789483061012
        Actual   Predicted
4         37.0   19.924475
10        17.0   36.825651
12        17.0   26.393538
13        28.0   44.140695
16        20.0   15.173344
...        ...         ...
462835    32.0   20.148838
462846    20.0   19.468397
462849    34.0   40.942524
462853   162.0  150.420688
462858    19.0   35.753132

[97182 rows x 2 columns]


In [0]:
#Using RandomForest
regressor = RandomForestRegressor(n_estimators = 100)
regressor.fit(train.drop(['Origin','Dest','FlightDate','date', 'ArrDel15','ArrDelayMinutes', 'ArrTime'], axis = 1), train_labels)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
y_pred = regressor.predict(test.drop(['Origin','Dest','FlightDate','date', 'ArrDel15','ArrDelayMinutes', 'ArrTime'], axis = 1))
print("RandomForest Mean Absolute Error: ", mean_absolute_error(test_labels, y_pred))
print("RandomForest Mean Squared Error: ", math.sqrt(mean_squared_error(test_labels, y_pred)))
#df_rf = pd.DataFrame({'Actual': test_labels, 'Predicted': y_pred})
#print(df_rf)
filename = path + '/drive/My Drive/ML Project/regressor_model_rf.sav'
pickle.dump(regressor, open(filename, 'wb'))


RandomForest Mean Absolute Error:  12.028666319257633
RandomForest Mean Squared Error:  17.011009036549996


In [0]:
#Using ExtraTrees
regressor_et = ExtraTreesRegressor(n_estimators = 100)
regressor_et.fit(train.drop(['Origin','Dest','FlightDate','date', 'ArrDel15','ArrDelayMinutes', 'ArrTime'], axis = 1), train_labels)

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=None, oob_score=False,
                    random_state=None, verbose=0, warm_start=False)

In [0]:
y_pred = regressor_et.predict(test.drop(['Origin','Dest','FlightDate','date', 'ArrDel15','ArrDelayMinutes','ArrTime'], axis = 1))
print("ExtraTrees Mean Absolute Error: ", mean_absolute_error(test_labels, y_pred))
print("ExtraTrees Mean Squared Error: ", math.sqrt(mean_squared_error(test_labels, y_pred)))
#df_et = pd.DataFrame({'Actual': test_labels, 'Predicted': y_pred})
#print(df_et)
filename = path + '/drive/My Drive/ML Project/regressor_model_et.sav'
pickle.dump(regressor_et, open(filename, 'wb'))

ExtraTrees Mean Absolute Error:  12.195099658372953
ExtraTrees Mean Squared Error:  17.283586063420838


In [0]:
#Using AdaBoost Regressor
regressor_ada = AdaBoostRegressor()
regressor_ada.fit(train.drop(['Origin','Dest','FlightDate','date', 'ArrDel15','ArrDelayMinutes','ArrTime'], axis = 1), train_labels)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=50, random_state=None)

In [0]:
y_pred = regressor_ada.predict(test.drop(['Origin','Dest','FlightDate','date', 'ArrDel15','ArrDelayMinutes','ArrTime'], axis = 1))
print("AdaBoost Mean Absolute Error: ", mean_absolute_error(test_labels, y_pred))
print("AdaBoost Mean Squared Error: ", math.sqrt(mean_squared_error(test_labels, y_pred)))
#df_ada = pd.DataFrame({'Actual': test_labels, 'Predicted': y_pred})
#print(df_ada)
filename = path + '/drive/My Drive/ML Project/regressor_model_ada.sav'
pickle.dump(regressor_ada, open(filename, 'wb'))

AdaBoost Mean Absolute Error:  19.318565348461163
AdaBoost Mean Squared Error:  24.334443433138617


In [0]:
#test_new = test.drop(['Origin','Dest','FlightDate','date', 'ArrTime'], axis = 1)
test_new = test[test['ArrDel15'] == 1]
#test_sorted = test_new.drop(['ArrDel15'], axis=1)
test_sorted = test_new.sort_values(['ArrDelayMinutes'])

In [0]:
lower = 15
mae = []
rmse = []
minutes = [15,100,200,500,1000]
for upper in minutes: 
  print("Iteration", lower)
  test_loop = pd.DataFrame()
  test_loop = test_sorted[(test_sorted['ArrDelayMinutes'] > lower) & (test_sorted['ArrDelayMinutes'] <= upper)]
  if (test_loop.empty != True):
    delay_minutes = regressor.predict(test_loop.drop(['Origin','Dest','FlightDate','date', 'ArrDel15', 'ArrDelayMinutes', 'ArrTime'], axis = 1))
    mae.append(mean_absolute_error(test_loop['ArrDelayMinutes'], delay_minutes))
    rmse.append(math.sqrt(mean_squared_error(test_loop['ArrDelayMinutes'], delay_minutes)))
    print("Appended")
  lower = upper

test_loop = test_sorted[test_sorted['ArrDelayMinutes'] > 1000]
delay_minutes = regressor.predict(test_loop.drop(['Origin','Dest','FlightDate','date', 'ArrDel15', 'ArrDelayMinutes', 'ArrTime'], axis = 1))
mae.append(mean_absolute_error(test_loop['ArrDelayMinutes'], delay_minutes))
rmse.append(math.sqrt(mean_squared_error(test_loop['ArrDelayMinutes'], delay_minutes)))
print(len(mae),len(rmse))


Iteration 15
Iteration 15
Appended
Iteration 100
Appended
Iteration 200
Appended
Iteration 500
Appended
5 5


In [0]:
print(rmse)

[14.120109256654612, 26.828421124500206, 29.44188209932818, 22.04866921449988, 42.8111064460513]
